In [1]:
import pandas as pd
import numpy as np

import itertools
import math

# from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy

import warnings
warnings.filterwarnings("ignore")

In [2]:
test_data = pd.read_csv('last_orders_subset.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5487 entries, 0 to 5486
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          5487 non-null   int64 
 1   SKU            5487 non-null   int64 
 2   Member         5487 non-null   object
 3   Delivery Date  5487 non-null   object
 4   Name           5487 non-null   object
dtypes: int64(2), object(3)
memory usage: 214.5+ KB


In [3]:
df = pd.read_csv('all_except_last_orders.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28984 entries, 0 to 28983
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          28984 non-null  int64 
 1   SKU            28984 non-null  int64 
 2   Member         28984 non-null  object
 3   Delivery Date  28984 non-null  object
 4   Name           28984 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


In [4]:
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'], dayfirst = True)

# Train Test Split

The data is split into last orders (for members who have more than 5 orders) and all but last order.
Previous orders are used to train the recommendation system and the last orders are used for testing.

## Last Order per Member

In [23]:
df.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [24]:
member_order_count = df[["Member", "Order"]].drop_duplicates()["Member"].value_counts().reset_index()
members_with_more_than_5_orders = member_order_count[member_order_count["count"] > 5]["Member"]
len(members_with_more_than_5_orders)

166

In [25]:
last_orders = df[df["Member"].isin(members_with_more_than_5_orders)][["Member", "Order", "Delivery Date"]] \
                .drop_duplicates() \
                .sort_values(["Order", "Delivery Date"], ascending = [True, False]) \
                .drop_duplicates(subset = "Member") \
                .reset_index(drop = True)["Order"]

In [26]:
len(last_orders)

166

In [27]:
df_train = df[~df["Order"].isin(last_orders)]
df_train.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [28]:
df_test = df[df["Order"].isin(last_orders)]
df_test.head()

,Order,SKU,Member,Delivery Date,Name
94,8101324,15668377,SSCEHNS,2014-03-15,Root Vegetables
95,8101324,15668684,SSCEHNS,2014-03-15,Beans
96,8101324,15669778,SSCEHNS,2014-03-15,Other Dals
97,8101324,15668521,SSCEHNS,2014-03-15,Bread
98,8101324,15669865,SSCEHNS,2014-03-15,Other Dals


In [29]:
# df_train.to_csv("df_train.csv", index = False)
# df_test.to_csv("df_test.csv", index = False)

## Simulating Forgotten Items

In [30]:
basket_list = list()
forgotten_list = list()

for order in df_test["Order"].unique():
    y_dummy = ['']*df_test[df_test["Order"] == order].shape[0]
    basket, forgotten, _, _ = train_test_split(df_test[df_test["Order"] == order], y_dummy, test_size = 5, random_state = 101)
    basket_list.append(basket)
    forgotten_list.append(forgotten)

test_basket_df = pd.concat(basket_list, ignore_index = True)
forgotten_items_df = pd.concat(forgotten_list, ignore_index = True)

In [31]:
test_basket_df["Order"].value_counts()

Order
7362753    26
7392553    20
7566535    19
7391877    16
7428900    16
           ..
7453262     3
7460103     3
7513528     3
7352666     3
8101324     3
Name: count, Length: 166, dtype: int64

In [32]:
forgotten_items_df["Order"].value_counts()

Order
8101324    5
7734044    5
7585543    5
7644998    5
7770723    5
          ..
7370908    5
7391877    5
7360479    5
7370657    5
7737427    5
Name: count, Length: 166, dtype: int64

In [33]:
# test_basket_df.to_csv("test_basket_df.csv", index = False)
# forgotten_items_df.to_csv("forgotten_items_df.csv", index = False)

# Item Based Collaborative Filtering

## SKU Co-occurrence Matrix

In [59]:
df_train = pd.read_csv("df_train.csv")
df_train.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [60]:
sku_list = list(df_train["SKU"].unique())
len(sku_list)

632

In [61]:
pairs = list(itertools.combinations_with_replacement(sku_list, 2))
print(len(pairs))

200028


In [62]:
%%time

s1_list = list()
s2_list = list()
cooccur_list = list()

for s1, s2 in pairs:
    s1_list.append(s1)
    s2_list.append(s2)
    
    freq = len(set(df_train[df_train["SKU"] == s1]["Order"]) \
               .intersection(set(df_train[df_train["SKU"] == s2]["Order"]))
              )
    cooccur_list.append(freq)

pairwise_cooccurrence = pd.DataFrame({"SKU1": s1_list, "SKU2": s2_list, "cooccurrence_frequency": cooccur_list})
pairwise_cooccurrence.head()

CPU times: user 25.9 s, sys: 57.6 ms, total: 26 s
Wall time: 26.2 s


,SKU1,SKU2,cooccurrence_frequency
0,15668375,15668375,108
1,15668375,15668467,33
2,15668375,15669863,13
3,15668375,15669778,11
4,15668375,15669767,4


In [63]:
%%time

co_matrix = pd.DataFrame(np.zeros((len(sku_list), len(sku_list))), index=sku_list, columns=sku_list)

for _, row in pairwise_cooccurrence.iterrows():
    co_matrix.loc[row["SKU1"], row["SKU2"]] = row["cooccurrence_frequency"]
    co_matrix.loc[row["SKU2"], row["SKU1"]] = row["cooccurrence_frequency"]

co_matrix.head()

CPU times: user 7.18 s, sys: 8.37 ms, total: 7.19 s
Wall time: 7.19 s


,15668375,15668467,15669863,15669778,15669767,15669832,15669772,15669970,15668478,15669861,...,7590868,15668382,7582051,7590866,7735524,15670172,7590864,92433757,92286348,7610713
15668375,108.0,33.0,13.0,11.0,4.0,4.0,11.0,3.0,27.0,12.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15668467,33.0,322.0,31.0,25.0,19.0,18.0,27.0,4.0,62.0,21.0,...,1.0,4.0,1.0,8.0,0.0,0.0,5.0,0.0,1.0,1.0
15669863,13.0,31.0,299.0,45.0,43.0,42.0,47.0,4.0,24.0,43.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
15669778,11.0,25.0,45.0,243.0,19.0,23.0,69.0,4.0,21.0,40.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
15669767,4.0,19.0,43.0,19.0,184.0,38.0,29.0,4.0,7.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Similarity Matrix

In [89]:
%%time

sim_matrix = pd.DataFrame(cosine_similarity(co_matrix), index=sku_list, columns=sku_list)
sim_matrix.head()

CPU times: user 14.5 ms, sys: 5.13 ms, total: 19.6 ms
Wall time: 15.5 ms


,15668375,15668467,15669863,15669778,15669767,15669832,15669772,15669970,15668478,15669861,...,7590868,15668382,7582051,7590866,7735524,15670172,7590864,92433757,92286348,7610713
15668375,1.000000,0.613503,0.409685,0.413580,0.310546,0.312635,0.414734,0.327831,0.548322,0.397129,...,0.208793,0.371777,0.342878,0.247329,0.175568,0.106815,0.259903,0.179621,0.269245,0.287959
15668467,0.613503,1.000000,0.448954,0.448271,0.363029,0.364380,0.459220,0.348781,0.609245,0.414204,...,0.347401,0.591794,0.320763,0.464254,0.190179,0.114182,0.427580,0.199862,0.406556,0.389189
15669863,0.409685,0.448954,1.000000,0.613309,0.632344,0.593434,0.623119,0.458031,0.410087,0.606846,...,0.325648,0.299858,0.408643,0.235254,0.120424,0.105860,0.234415,0.258891,0.269451,0.185499
15669778,0.413580,0.448271,0.613309,1.000000,0.512776,0.523431,0.721877,0.443519,0.417328,0.605837,...,0.338942,0.307762,0.322747,0.241307,0.126258,0.110800,0.296229,0.365097,0.243480,0.200260
15669767,0.310546,0.363029,0.632344,0.512776,1.000000,0.631637,0.571371,0.486398,0.301178,0.587039,...,0.272512,0.213498,0.293450,0.163419,0.099506,0.107323,0.176269,0.226608,0.242507,0.137267


## Test on df_test

In [90]:
test_basket_df = pd.read_csv("test_basket_df.csv")

In [91]:
# recc_df = pd.DataFrame(columns = ["Order", "SKU", "Member", "Similarity Score"])

# for order in test_basket_df["Order"].unique():
#     test_order = test_basket_df[test_basket_df["Order"] == order]
#     test_recc = pd.DataFrame(columns = ["SKU", "Similarity Score"])
#     for sku in test_order["SKU"]:
#         df_ = pd.DataFrame(sim_matrix.loc[sku]).reset_index()  
#         df_.columns = ["SKU", "Similarity Score"]
#         df_ = df_.sort_values("Similarity Score", ascending = False).reset_index(drop = True).iloc[1:]
#         test_recc = pd.concat([test_recc, df_]) 
#     test_recc["Order"] = order
#     test_recc["Member"] = test_order["Member"].unique()[0]
#     test_recc = test_recc[["Order", "SKU", "Member", "Similarity Score"]] \
#                     .sort_values("Similarity Score", ascending = False) \
#                     .iloc[:5]
#                     # .sort_values(["SKU", "Similarity Score"], ascending = [True, False]) \
#                     # .drop_duplicates(subset = "SKU") \
                    

#     recc_df = pd.concat([recc_df, test_recc], ignore_index = True)

In [92]:
recc_df = pd.DataFrame(columns = ["Order", "SKU", "Member"])

for order in test_basket_df["Order"].unique():
    test_order = test_basket_df[test_basket_df["Order"] == order]
    test_recc = pd.DataFrame(columns = ["SKU", "Similarity Score"])
    for sku in test_order["SKU"]:
        df_ = pd.DataFrame(sim_matrix.loc[sku]).reset_index()
        df_.columns = ["SKU", "Similarity Score"]
        df_ = df_.sort_values("Similarity Score", ascending = False).reset_index(drop = True).iloc[1:]
        test_recc = pd.concat([test_recc, df_])
        # recc_count = 0
        # for i in range(1, 632):
        #     if df_.iloc[i]["SKU"] in list(test_recc["SKU"]):
        #         pass
        #     else:
        #         test_recc = pd.concat([test_recc, df_.iloc[i:i+1]]) 
        #         recc_count += 1
            
        #     if recc_count >= 3:
        #         break    
    test_recc = test_recc \
                .sort_values("Similarity Score", ascending = False) \
                .drop_duplicates(subset = "SKU") \
                .iloc[:5]
    test_recc["Order"] = order
    test_recc["Member"] = test_order["Member"].unique()[0]
    test_recc = test_recc[["Order", "SKU", "Member"]]

    recc_df = pd.concat([recc_df, test_recc], ignore_index = True)

In [93]:
recc_df.head()

,Order,SKU,Member
0,8101324,15668460,SSCEHNS
1,8101324,15668381,SSCEHNS
2,8101324,15668467,SSCEHNS
3,8101324,21409124,SSCEHNS
4,8101324,15668455,SSCEHNS


## Evaluation using Recall@5 Metric

In [94]:
forgotten_items_df = pd.read_csv("forgotten_items_df.csv")
forgotten_items_df.head()

,Order,SKU,Member,Delivery Date,Name
0,8101324,15668377,SSCEHNS,2014-03-15,Root Vegetables
1,8101324,15669778,SSCEHNS,2014-03-15,Other Dals
2,8101324,15669814,SSCEHNS,2014-03-15,Jaggery
3,8101324,15669865,SSCEHNS,2014-03-15,Other Dals
4,8101324,15669970,SSCEHNS,2014-03-15,Cashews


In [95]:
recall_list = list()

for order in list(forgotten_items_df["Order"].unique()):
    forgotten_SKUs = forgotten_items_df[forgotten_items_df["Order"] == order]["SKU"]
    recc_SKUs = recc_df[recc_df["Order"] == order]["SKU"]
    recall = len(set(forgotten_SKUs).intersection(set(recc_SKUs)))/len(set(forgotten_SKUs))
    recall_list.append(recall)

In [96]:
np.mean(recall_list)

0.10963855421686744

## Prepare Kaggle Submission    

In [47]:
%%time

sku_list = list(df["SKU"].unique())
pairs = list(itertools.combinations_with_replacement(sku_list, 2))

s1_list = list()
s2_list = list()
cooccur_list = list()

for s1, s2 in pairs:
    s1_list.append(s1)
    s2_list.append(s2)
    
    freq = len(set(df[df["SKU"] == s1]["Order"]) \
               .intersection(set(df[df["SKU"] == s2]["Order"]))
              )
    cooccur_list.append(freq)

pairwise_cooccurrence = pd.DataFrame({"SKU1": s1_list, "SKU2": s2_list, "cooccurrence_frequency": cooccur_list})

co_matrix = pd.DataFrame(np.zeros((len(sku_list), len(sku_list))), index=sku_list, columns=sku_list)

for _, row in pairwise_cooccurrence.iterrows():
    co_matrix.loc[row["SKU1"], row["SKU2"]] = row["cooccurrence_frequency"]
    co_matrix.loc[row["SKU2"], row["SKU1"]] = row["cooccurrence_frequency"]

sim_matrix = pd.DataFrame(cosine_similarity(co_matrix), index=sku_list, columns=sku_list)

recc_df = pd.DataFrame(columns = ["Order", "SKU", "Member"])

for order in test_data["Order"].unique():
    test_order = test_data[test_data["Order"] == order]
    test_recc = pd.DataFrame(columns = ["SKU", "Similarity Score"])
    for sku in test_order["SKU"]:
        try:
            df_ = pd.DataFrame(sim_matrix.loc[sku]).reset_index()
            df_.columns = ["SKU", "Similarity Score"]
            df_ = df_.sort_values("Similarity Score", ascending = False).reset_index(drop = True)
            recc_count = 0
            for i in range(1, 632):
                if df_.iloc[i]["SKU"] in list(test_recc["SKU"]):
                    pass
                else:
                    test_recc = pd.concat([test_recc, df_.iloc[i:i+1]]) 
                    recc_count += 1
                
                if recc_count >= 3:
                    break
        except:
            pass
    test_recc = test_recc.sort_values("Similarity Score", ascending = False).iloc[:5]
    test_recc["Order"] = order
    test_recc["Member"] = test_order["Member"].unique()[0]
    test_recc = test_recc[["Order", "SKU", "Member"]]

    recc_df = pd.concat([recc_df, test_recc], ignore_index = True)

recc_df.head()

CPU times: user 45.1 s, sys: 53 ms, total: 45.1 s
Wall time: 45.1 s


,Order,SKU,Member
0,7409204,15668465,SWLCNOE
1,7409204,15669772,SWLCNOE
2,7409204,15668467,SWLCNOE
3,7409204,15668379,SWLCNOE
4,7409204,15668688,SWLCNOE


In [48]:
recc_df = recc_df[["Order", "SKU", "Member"]].reset_index().rename(columns = {"index": "ID"})
recc_df["ID"] = recc_df["ID"].apply(lambda x: x+1)
recc_df.head()

,ID,Order,SKU,Member
0,1,7409204,15668465,SWLCNOE
1,2,7409204,15669772,SWLCNOE
2,3,7409204,15668467,SWLCNOE
3,4,7409204,15668379,SWLCNOE
4,5,7409204,15668688,SWLCNOE


In [49]:
# recc_df.to_csv("item_based_collab_filtering_recc.csv", index = False)